## Importing libraries

In [31]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Contacting API and reading our token

In [32]:
print("reading csv")
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

print("contacting API")
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

reading csv
contacting API


{'companyName': 'Apple Inc',
 'marketcap': 2422469212290,
 'week52high': 162.97,
 'week52low': 108.35,
 'week52highSplitAdjustOnly': 158.02,
 'week52lowSplitAdjustOnly': 112.4,
 'week52change': 0.23028464064448853,
 'sharesOutstanding': 16972813398,
 'float': 0,
 'avg10Volume': 79131088,
 'avg30Volume': 86922079,
 'day200MovingAvg': 140.48,
 'day50MovingAvg': 150.6,
 'employees': 153663,
 'ttmEPS': 5.3,
 'ttmDividendRate': 0.870402555659621,
 'dividendYield': 0.006042825586981832,
 'nextDividendDate': '',
 'exDividendDate': '2021-07-28',
 'nextEarningsDate': '2021-10-26',
 'peRatio': 28.37674745641887,
 'beta': 1.3910303930963228,
 'maxChangePercent': 56.47677113252396,
 'year5ChangePercent': 4.457352564952932,
 'year2ChangePercent': 1.525803169746812,
 'year1ChangePercent': 0.23240621507644255,
 'ytdChangePercent': 0.08368869629775158,
 'month6ChangePercent': 0.07979939701554509,
 'month3ChangePercent': -0.01397356957824421,
 'month1ChangePercent': -0.040812726021838,
 'day30ChangePer

## Splitting list of stocks in to groups of 100 so you can use batch api calls, which are significantly faster than the alternative.

In [34]:
data['year1ChangePercent']

print("defining group API call logic")
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

defining group API call logic


## Creating dataframe

In [35]:
print("creating pandas dataframe")
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']


final_dataframe = pd.DataFrame(columns = my_columns)
print("done")

creating pandas dataframe
done


## Populating dataframe with API data.

In [36]:
print("populating pandas dataframe with API data (SLOOOOOOW)")
for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

populating pandas dataframe with API data (SLOOOOOOW)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,156.30,0.462653,N/A
1,AAL,20.82,0.539015,N/A
2,AAP,221.53,0.412471,N/A
3,AAPL,147.80,0.236826,N/A
4,ABBV,112.28,0.346112,N/A
5,ABC,124.59,0.256864,N/A
6,ABMD,344.91,0.24065,N/A
7,ABT,123.30,0.0961836,N/A
8,ACN,328.10,0.448564,N/A
9,ADBE,589.56,0.154765,N/A


## Sorting dataframe, then cutting off everything below 50.

In [37]:
print("sorting dataframe for right values, then cutting everything below 50th place off")
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

sorting dataframe for right values, then cutting everything below 50th place off


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DVN,39.99,3.20729,N/A
1,MRO,16.41,2.85371,N/A
2,FANG,110.60,2.74346,N/A
3,LB,81.45,2.37504,N/A
4,OXY,34.07,2.18379,N/A
5,MCHP,147.53,1.66046,N/A
6,EOG,93.84,1.53027,N/A
7,SIVB,684.21,1.47122,N/A
8,FTNT,320.00,1.45936,N/A
9,IT,311.80,1.42054,N/A


## Creating a function to take in user-input portfolio value.

In [38]:
print("portfolio input logic...")
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

# portfolio_input()

portfolio_size = 10000000

print(portfolio_size)
print("done")

portfolio input logic...
10000000
done


## Creating a more refined selection of high value stock that consider how long something has been gaining value.

In [39]:
print("creating a more refined high level selection of high value stocks (stocks that have been accumulating value over a longer perdiod of time)")
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

creating a more refined high level selection of high value stocks (stocks that have been accumulating value over a longer perdiod of time)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DVN,39.99,3.20729,4903
1,MRO,16.41,2.85371,11948
2,FANG,110.60,2.74346,1772
3,LB,81.45,2.37504,2407
4,OXY,34.07,2.18379,5755
5,MCHP,147.53,1.66046,1329
6,EOG,93.84,1.53027,2089
7,SIVB,684.21,1.47122,286
8,FTNT,320.00,1.45936,612
9,IT,311.80,1.42054,628


## Creating dataframe for new algorithm.

In [40]:
print("creating hqm dataframe")
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

print("done")

creating hqm dataframe
done


## Populating dataframe with API data

In [41]:
print("populating dataframe with API data. Will take a while.")

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns
hqm_dataframe

populating dataframe with API data. Will take a while.


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,153.74,N/A,0.468196,N/A,0.162621,N/A,0.0180899,N/A,-0.145202,N/A,N/A
1,AAL,20.20,N/A,0.539776,N/A,-0.15469,N/A,-0.0397499,N/A,0.0604292,N/A,N/A
2,AAP,223.56,N/A,0.409283,N/A,0.180651,N/A,0.00837298,N/A,0.0778955,N/A,N/A
3,AAPL,144.60,N/A,0.230365,N/A,0.0793789,N/A,-0.014086,N/A,-0.0410462,N/A,N/A
4,ABBV,112.79,N/A,0.341447,N/A,0.0597203,N/A,-0.0365557,N/A,0.0435491,N/A,N/A
5,ABC,124.46,N/A,0.263898,N/A,0.0306942,N/A,0.0411579,N/A,-0.0289985,N/A,N/A
6,ABMD,345.31,N/A,0.240825,N/A,0.0101102,N/A,0.0387499,N/A,-0.0674152,N/A,N/A
7,ABT,118.64,N/A,0.0991693,N/A,-0.0139513,N/A,-0.00784556,N/A,-0.0813632,N/A,N/A
8,ACN,333.21,N/A,0.453352,N/A,0.142564,N/A,0.0443985,N/A,-0.0511627,N/A,N/A
9,ADBE,581.25,N/A,0.15439,N/A,0.150641,N/A,-0.0477172,N/A,-0.130345,N/A,N/A


## This is where the troubleshooting started because further down a block of code threw a a NoneType error.

Components:
- pd.set_option >> makes pandas print out every row, rather than just first and last few.
- Bunch of print statements to see the types of data. This is how I found out there were a few NoneType values hidden in there, throwing the code.

In [50]:
pd.set_option("display.max_rows", 50, "display.max_columns", None)

# start = 401
# finish = 504

# print("Ticker, Price, Number of Shares to Buy, One-Year Price Return")
# for i in range(start, finish):
#     row = i
#     print(f"row {row}:{type(hqm_dataframe['Ticker'][row])}, {type(hqm_dataframe['Price'][row])}, {type(hqm_dataframe['Number of Shares to Buy'][row])}, {type(hqm_dataframe['One-Year Price Return'][row])}")

# print("\n\n")

# print("One-Year Return Percentile, Six-Month Price Return, Six-Month Return Percentile")  
# for i in range(start, finish):
#     row = i
#     print(f"row {row}:{type(hqm_dataframe['One-Year Return Percentile'][row])}, {type(hqm_dataframe['Six-Month Price Return'][row])}, {type(hqm_dataframe['Six-Month Return Percentile'][row])}")

# print("\n\n")       
    
# print("Three-Month Price Return, Three-Month Return Percentile, One-Month Price Return, One-Month Return Percentile")  
# for i in range(start, finish):
#     row = i
#     print(f"row {row}:{type(hqm_dataframe['Three-Month Price Return'][row])}, {type(hqm_dataframe['Three-Month Return Percentile'][row])}, {type(hqm_dataframe['One-Month Price Return'][row])}, {type(hqm_dataframe['One-Month Return Percentile'][row])}")


## Further testing and confirming the NoneType values.

In [46]:
# #testcase
# print(type(hqm_dataframe['One-Year Price Return'][100]))
# print(type(hqm_dataframe['Six-Month Price Return'][100]))
# print(type(hqm_dataframe['Three-Month Price Return'][100]))
# print(type(hqm_dataframe['One-Month Price Return'][100]))

# #known to return NoneType
# print(type(hqm_dataframe['One-Year Price Return'][118]))
# print(type(hqm_dataframe['Six-Month Price Return'][118]))
# print(type(hqm_dataframe['Three-Month Price Return'][118]))
# print(type(hqm_dataframe['One-Month Price Return'][118]))

# print(type(hqm_dataframe['One-Year Price Return'][165]))
# print(type(hqm_dataframe['Six-Month Price Return'][165]))
# print(type(hqm_dataframe['Three-Month Price Return'][165]))
# print(type(hqm_dataframe['One-Month Price Return'][165]))

# print(type(hqm_dataframe['One-Year Price Return'][326]))
# print(type(hqm_dataframe['Six-Month Price Return'][326]))
# print(type(hqm_dataframe['Three-Month Price Return'][326]))
# print(type(hqm_dataframe['One-Month Price Return'][326]))

# print(type(hqm_dataframe['One-Year Price Return'][327]))
# print(type(hqm_dataframe['Six-Month Price Return'][327]))
# print(type(hqm_dataframe['Three-Month Price Return'][327]))
# print(type(hqm_dataframe['One-Month Price Return'][327]))

## This line of code was the solution. It fills all NoneType values as NaN values (a weird type of float if I'm not mistaken)

In [48]:
hqm_dataframe = hqm_dataframe.fillna(value=np.nan)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,153.74,N/A,0.468196,N/A,0.162621,N/A,0.018090,N/A,-0.145202,N/A,N/A
1,AAL,20.20,N/A,0.539776,N/A,-0.154690,N/A,-0.039750,N/A,0.060429,N/A,N/A
2,AAP,223.56,N/A,0.409283,N/A,0.180651,N/A,0.008373,N/A,0.077896,N/A,N/A
3,AAPL,144.60,N/A,0.230365,N/A,0.079379,N/A,-0.014086,N/A,-0.041046,N/A,N/A
4,ABBV,112.79,N/A,0.341447,N/A,0.059720,N/A,-0.036556,N/A,0.043549,N/A,N/A
5,ABC,124.46,N/A,0.263898,N/A,0.030694,N/A,0.041158,N/A,-0.028999,N/A,N/A
6,ABMD,345.31,N/A,0.240825,N/A,0.010110,N/A,0.038750,N/A,-0.067415,N/A,N/A
7,ABT,118.64,N/A,0.099169,N/A,-0.013951,N/A,-0.007846,N/A,-0.081363,N/A,N/A
8,ACN,333.21,N/A,0.453352,N/A,0.142564,N/A,0.044399,N/A,-0.051163,N/A,N/A
9,ADBE,581.25,N/A,0.154390,N/A,0.150641,N/A,-0.047717,N/A,-0.130345,N/A,N/A


## This is the code snippet that caused the error. It runs fine now with above solution.
Basically it doesn't like it if you try to do any kind of sorting and there are NoneType values in there.

In [52]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:

        # THE PROBLEM LINE
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
# for time_period in time_periods:
#     print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe


# print("if you made it here then wow you fixed one problem ")

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,153.74,N/A,0.468196,0.683168,0.162621,0.760396,0.018090,0.568317,-0.145202,0.0118812,N/A
1,AAL,20.20,N/A,0.539776,0.726733,-0.154690,0.0792079,-0.039750,0.29901,0.060429,0.790099,N/A
2,AAP,223.56,N/A,0.409283,0.615842,0.180651,0.79802,0.008373,0.520792,0.077896,0.837624,N/A
3,AAPL,144.60,N/A,0.230365,0.378218,0.079379,0.576238,-0.014086,0.394059,-0.041046,0.334653,N/A
4,ABBV,112.79,N/A,0.341447,0.532673,0.059720,0.522772,-0.036556,0.312871,0.043549,0.740594,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,124.09,N/A,0.323897,0.506931,0.084953,0.586139,0.046607,0.69703,-0.051045,0.263366,N/A
501,ZBH,153.46,N/A,-0.002980,0.112871,-0.137693,0.0950495,-0.079206,0.158416,0.046090,0.744554,N/A
502,ZBRA,519.26,N/A,0.761721,0.857426,-0.018144,0.29703,-0.090515,0.136634,-0.145248,0.00990099,N/A
503,ZION,64.80,N/A,1.117816,0.946535,0.156042,0.744554,0.228107,0.962376,0.106857,0.889109,N/A


## Next few blocks do the sorting magic with the statistics library.


In [53]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [55]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,153.74,N/A,0.468196,0.683168,0.162621,0.760396,0.018090,0.568317,-0.145202,0.0118812,0.505941
1,AAL,20.20,N/A,0.539776,0.726733,-0.154690,0.0792079,-0.039750,0.29901,0.060429,0.790099,0.473762
2,AAP,223.56,N/A,0.409283,0.615842,0.180651,0.79802,0.008373,0.520792,0.077896,0.837624,0.693069
3,AAPL,144.60,N/A,0.230365,0.378218,0.079379,0.576238,-0.014086,0.394059,-0.041046,0.334653,0.420792
4,ABBV,112.79,N/A,0.341447,0.532673,0.059720,0.522772,-0.036556,0.312871,0.043549,0.740594,0.527228
...,...,...,...,...,...,...,...,...,...,...,...,...
46,ARE,201.56,N/A,0.196018,0.338614,0.157939,0.748515,0.016082,0.558416,-0.031062,0.394059,0.509901
47,ATO,90.13,N/A,-0.051933,0.0772277,-0.083960,0.162376,-0.084124,0.152475,-0.034420,0.368317,0.190099
48,ATVI,80.55,N/A,-0.002893,0.114851,-0.186513,0.0455446,-0.161331,0.0376238,-0.026694,0.419802,0.154455
49,AVB,231.40,N/A,0.465166,0.677228,0.224725,0.865347,0.021145,0.580198,0.008841,0.613861,0.684158


In [24]:
portfolio_input()

Enter the value of your portfolio:10000000


In [58]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,153.74,1275,0.468196,0.683168,0.162621,0.760396,0.018090,0.568317,-0.145202,0.0118812,0.505941
1,AAL,20.20,9706,0.539776,0.726733,-0.154690,0.0792079,-0.039750,0.29901,0.060429,0.790099,0.473762
2,AAP,223.56,877,0.409283,0.615842,0.180651,0.79802,0.008373,0.520792,0.077896,0.837624,0.693069
3,AAPL,144.60,1356,0.230365,0.378218,0.079379,0.576238,-0.014086,0.394059,-0.041046,0.334653,0.420792
4,ABBV,112.79,1738,0.341447,0.532673,0.059720,0.522772,-0.036556,0.312871,0.043549,0.740594,0.527228
...,...,...,...,...,...,...,...,...,...,...,...,...
46,ARE,201.56,972,0.196018,0.338614,0.157939,0.748515,0.016082,0.558416,-0.031062,0.394059,0.509901
47,ATO,90.13,2175,-0.051933,0.0772277,-0.083960,0.162376,-0.084124,0.152475,-0.034420,0.368317,0.190099
48,ATVI,80.55,2434,-0.002893,0.114851,-0.186513,0.0455446,-0.161331,0.0376238,-0.026694,0.419802,0.154455
49,AVB,231.40,847,0.465166,0.677228,0.224725,0.865347,0.021145,0.580198,0.008841,0.613861,0.684158


## From here on it is all about writing and saving to Excel.

In [26]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [27]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [28]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [29]:
writer.save()